In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from tqdm import tqdm

tqdm.pandas()

In [ ]:
from backtesting import Backtest, Strategy

from utils.signals import LONG, SHORT

class StrategyBacktest(Strategy):
    
    # Based on https://www.youtube.com/watch?v=J6VRMhDnVrM

    size = 0.1
    slperc = 0.02
    tpperc = 0.04

    def init(self):
        super().init()
    
    def next(self):
        super().next()
        df = self.data.df
        signal = df.loc[self.data.index[-1], "TotalSignal"]
        close = self.data.Close[-1]
        
        if signal == LONG and not self.position:
            self.buy(size=self.size,
                     sl=close - self.slperc * close,
                     tp=close + self.tpperc * close)
        elif signal == SHORT and not self.position:
            self.sell(size=self.size,
                      sl=close + self.slperc * close,
                      tp=close - self.tpperc * close)

/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [36]:
from utils.asset import Asset
from utils.strategy import Strategy as TradingStrategy
# Avoid naming conflict with the backtesting library's Strategy

def run_backtest(trading_strategy: TradingStrategy, asset: Asset):
    strat = trading_strategy()
    df = asset.apply_strategy(strat)
    bt = Backtest(df, StrategyBacktest, cash=5000, margin=1/10, commission=0.002)
    results = bt.optimize(slperc=[i/100 for i in range(1, 8)],
                                 tpperc=[i/100 for i in range(1, 8)],
                                 maximize='Return [%]',
                                 max_tries=3000,
                                 random_state=0)
    bt.plot()
    return results

In [34]:
from utils.portfolio import Portfolio
from strategies.strategy02 import Strategy02
from utils.tickers import tickers

portfolio = Portfolio(tickers=tickers, start="2022-01-01", end="2024-12-31")
nvda = portfolio.get_asset("NVDA")

In [40]:
results = run_backtest(Strategy02, nvda)
print(results)

Backtest.optimize:   0%|          | 0/49 [00:00<?, ?it/s]/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients witho

KeyboardInterrupt: 